###### Our aim in this notebook is extracting the data in https://www.euroncap.com/en/results/ds/9/46602 and store it in a CSV file

In [1]:
#import libraries

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import pandas as pd
import numpy as np


In [2]:
# specify cols names

INDEX_LIST = ['Rate', 'Adult Occupant', 'Child Occupant', 'Vulnerable Road Users', 'Safety Assist', 'Tested Model', 'Body Type', 'Year Of Publication', 'Kerb Weight', 'VIN From Which Rating Applies', 'Class',
                'Front Airbag-Driver', 'Belt Pretensioner-Driver', 'Belt Loadlimiter-Driver', 'Knee Airbag-Driver', 'Side Head Airbag-Driver', 'Side Chest Airbag-Driver', 'Side Pelvis Airbag-Driver', 'Centre Airbag-Driver', 'Isofix/i-Size-Driver', 'Integrated Child Seat-Driver', 'Airbag cut-off switch-Driver', 'Seatbelt Reminder-Driver',
                'Front Airbag-Passenger', 'Belt Pretensioner-Passenger', 'Belt Loadlimiter-Passenger', 'Knee Airbag-Passenger', 'Side Head Airbag-Passenger', 'Side Chest Airbag-Passenger', 'Side Pelvis Airbag-Passenger', 'Centre Airbag-Passenger', 'Isofix/i-Size-Passenger', 'Integrated Child Seat-Passenger', 'Airbag cut-off switch-Passenger', 'Seatbelt Reminder-Passenger',
                'Front Airbag-Rear', 'Belt Pretensioner-Rear', 'Belt Loadlimiter-Rear', 'Knee Airbag-Rear', 'Side Head Airbag-Rear', 'Side Chest Airbag-Rear', 'Side Pelvis Airbag-Rear', 'Centre Airbag-Rear', 'Isofix/i-Size-Rear', 'Integrated Child Seat-Rear', 'Airbag cut-off switch-Rear', 'Seatbelt Reminder-Rear',
                'Active Bonnet', 'AEB Vulnerable Road Users', 'AEB Pedestrian - Reverse', 'AEB Car-to-Car', 'Speed Assistance', 'Lane Assist System']



In [3]:
len(INDEX_LIST)

53

In [4]:
# open Euro NCAP website
browser = webdriver.Chrome()
browser.get('https://www.euroncap.com/en/ratings-rewards/latest-safety-ratings/#?selectedMake=0&selectedMakeName=Select%20a%20make&selectedModel=0&selectedStar=&includeFullSafetyPackage=true&includeStandardSafetyPackage=true&selectedModelName=All&selectedProtocols=49446,45155,41776,40302,34803,30636,26061,24370&selectedClasses=1202,1199,1201,1196,1205,1203,1198,1179,40250,1197,1204,1180,34736,44997&allClasses=true&allProtocols=false&allDriverAssistanceTechnologies=false&selectedDriverAssistanceTechnologies=&thirdRowFitment=false')

try:
    elem = WebDriverWait(browser, 30).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="rating-table1"]/div[2]/div[2]/div[1]')) #This is a dummy element
)

    # Extract cars' pages links 
    rows  = browser.find_elements(By.CSS_SELECTOR, ".rating-table-row-c.c9")
    links =[link.find_element(By.TAG_NAME, 'a').get_attribute('href') for link in rows]
    
finally:
    browser.quit()




In [5]:
len(links)


302

In [6]:
DATA = np.array([INDEX_LIST])
DATA

array([['Rate', 'Adult Occupant', 'Child Occupant',
        'Vulnerable Road Users', 'Safety Assist', 'Tested Model',
        'Body Type', 'Year Of Publication', 'Kerb Weight',
        'VIN From Which Rating Applies', 'Class', 'Front Airbag-Driver',
        'Belt Pretensioner-Driver', 'Belt Loadlimiter-Driver',
        'Knee Airbag-Driver', 'Side Head Airbag-Driver',
        'Side Chest Airbag-Driver', 'Side Pelvis Airbag-Driver',
        'Centre Airbag-Driver', 'Isofix/i-Size-Driver',
        'Integrated Child Seat-Driver', 'Airbag cut-off switch-Driver',
        'Seatbelt Reminder-Driver', 'Front Airbag-Passenger',
        'Belt Pretensioner-Passenger', 'Belt Loadlimiter-Passenger',
        'Knee Airbag-Passenger', 'Side Head Airbag-Passenger',
        'Side Chest Airbag-Passenger', 'Side Pelvis Airbag-Passenger',
        'Centre Airbag-Passenger', 'Isofix/i-Size-Passenger',
        'Integrated Child Seat-Passenger',
        'Airbag cut-off switch-Passenger', 'Seatbelt Reminder-Passe

In [7]:
fiveStars = 'https://www.euroncap.com/gfx/stars5-165x24.png'
fourStars = 'https://www.euroncap.com/gfx/stars4-165x24.png'
threeStars = 'https://www.euroncap.com/gfx/stars3-165x24.png'
twoStars = 'https://www.euroncap.com/gfx/stars2-165x24.png'
oneStar = 'https://www.euroncap.com/gfx/stars1-165x24.png'


Fitted_Standard = 'https://www.euroncap.com/gfx/picto_standard.svg'
Fitted_Safety_Pack = 'https://www.euroncap.com/gfx/picto_safety.svg' 
Not_Fitted = 'https://www.euroncap.com/gfx/picto_not-safety.svg'
Not_Available = 'https://www.euroncap.com/gfx/picto_not-available.svg'
Not_Applicable = 'https://www.euroncap.com/gfx/picto_not-applicable.svg'


In [8]:
def setValue( link ):
    if link == Fitted_Standard:
        return 'Fitted_Standard'
    elif link == Fitted_Safety_Pack : 
        return 'Fitted_Safety_Pack'
    elif link == Not_Fitted : 
        return 'Not_Fitted'
    elif link == Not_Available : 
        return 'Not_Available'
    elif link == Not_Applicable : 
        return 'Not_Applicable'
    else :
        return 'nan'

In [9]:
def setRate(stars):
    if stars == fiveStars:
        return 5
    elif stars == fourStars:
        return 4
    elif stars == threeStars:
        return 3
    elif stars == twoStars:
        return 2
    elif stars == oneStar:
        return 1
    else :
        return 0

In [15]:
x = 0

for link in links:
    try:

        Cars_col  = [];
        Cars_val  = [];


        browser = webdriver.Chrome()
        action = ActionChains(browser)
        browser.get(link)


        # -- Rate ---
        stars = browser.find_element(By.CSS_SELECTOR,'.stars').find_element(By.TAG_NAME, 'img').get_attribute('src')
        rate = setRate(stars)
        Cars_col.append('Rate')
        Cars_val.append(rate)

        # -- Specification --

        SPECEIFICATION = [cls.text for cls in browser.find_elements(By.CSS_SELECTOR,'.tcol1')]
        SPEC_VALUES = [value.text for value in browser.find_elements(By.CSS_SELECTOR,'.tcol2')]

        Cars_col.extend(SPECEIFICATION)
        Cars_val.extend(SPEC_VALUES)


        # -- Adult Occupant , Child Occupant, Vulnerable Road Users, MoreSafety Assist

        detailsSection = browser.find_element(By.XPATH, '//*[@id="site-canvas"]/div[4]/div/div[2]/div[8]')

        for title in detailsSection.find_elements(By.CSS_SELECTOR , '.rating-title' ):
            try: 
                Cars_col.append(title.find_element(By.TAG_NAME, 'p').text)
            except:
                Cars_col.append(np.nan)


        for value in browser.find_elements(By.CSS_SELECTOR,'.value'):
                Cars_val.append(value.text)

        
        # -- safety table --

        speTab = browser.find_element(By.XPATH, '//*[@id="car-rating-details"]/ul/li[2]')
        actions = ActionChains(browser)
        actions.click(speTab)
        actions.perform()


        try:
            
            flg = True

            try :
                safetyTable = browser.find_element(By.XPATH, '//*[@id="tab2"]/div[1]/div[2]')
            except Exception.NoSuchElementException as e:
                print(e)



            while flg:
                SAFETY_EQ_C1 = [value.text for value in safetyTable.find_elements(By.CSS_SELECTOR,'.table-row-c.c1')]

                # print('SAFETY_EQ_C1: ', SAFETY_EQ_C1)

                length = [len(titel) for titel in SAFETY_EQ_C1 ]

                if 0 not in length:
                    flg = False




            i =0 
            for v in safetyTable.find_elements(By.CSS_SELECTOR,'.table-row-c.c2'):
                col = SAFETY_EQ_C1[i]+'-Driver'
                Cars_col.append(col)
                try: 
                    valueImg = v.find_element(By.TAG_NAME, 'img').get_attribute('src')
                    Cars_val.append(setValue(valueImg))
                except: 
                    Cars_val.append(np.nan)
                
                i = i+1


            i =0
            for v in safetyTable.find_elements(By.CSS_SELECTOR,'.table-row-c.c3'):
                col = SAFETY_EQ_C1[i]+'-Passenger'
                Cars_col.append(col)
                try: 
                    valueImg = v.find_element(By.TAG_NAME, 'img').get_attribute('src')
                    Cars_val.append(setValue(valueImg))
                except: 
                    Cars_val.append(np.nan)
                i = i+1


            i =0
            for v in safetyTable.find_elements(By.CSS_SELECTOR,'.table-row-c.c4'):
                col = SAFETY_EQ_C1[i]+'-Rear'
                Cars_col.append(col)
                try: 
                    valueImg = v.find_element(By.TAG_NAME, 'img').get_attribute('src')
                    Cars_val.append(setValue(valueImg))
                except: 
                    Cars_val.append(np.nan)


                i = i+1



            otherSafetyTable = browser.find_element(By.XPATH, '//*[@id="tab2"]/div[2]')

            otherSafetyTitle = [value.text for value in otherSafetyTable.find_elements(By.CSS_SELECTOR,'.table-row-c.c1')]

            i =0 
            for v in otherSafetyTable.find_elements(By.CSS_SELECTOR,'.table-row-c.c2'):
                col = otherSafetyTitle[i]
                Cars_col.append(col)
                try: 
                    valueImg = v.find_element(By.TAG_NAME, 'img').get_attribute('src')
                    Cars_val.append(setValue(valueImg))
                except: 
                    Cars_val.append(np.nan)
                i = i+1
                


        except Exception as e:
            print(e)
            

                
        # transforming the extracted data into a structured format

        print(Cars_col, '\n', Cars_val)    
        INFO = dict(zip(Cars_col,Cars_val))
        carInfo = []
        for cls in INDEX_LIST:
            if cls in INFO.keys():
                carInfo.append(INFO[cls])
            else:
                carInfo.append(np.nan)
        
        DATA = np.append(DATA,[carInfo], axis=0)

    except Exception as e :
        print(e)
    finally:
        browser.quit()




['Rate', 'Tested Model', 'Body Type', 'Year Of Publication', 'Kerb Weight', 'VIN From Which Rating Applies', 'Class', 'Adult Occupant', 'Child Occupant', 'Vulnerable Road Users', 'Safety Assist', 'FRONTAL CRASH PROTECTION-Driver', 'Front Airbag-Driver', 'Belt Pretensioner-Driver', 'Belt Loadlimiter-Driver', 'Knee Airbag-Driver', 'LATERAL CRASH PROTECTION-Driver', 'Side Head Airbag-Driver', 'Side Chest Airbag-Driver', 'Side Pelvis Airbag-Driver', 'Centre Airbag-Driver', 'CHILD PROTECTION-Driver', 'Isofix/i-Size-Driver', 'Integrated Child Seat-Driver', 'Airbag cut-off switch-Driver', 'SAFETY ASSIST-Driver', 'Seatbelt Reminder-Driver', 'FRONTAL CRASH PROTECTION-Passenger', 'Front Airbag-Passenger', 'Belt Pretensioner-Passenger', 'Belt Loadlimiter-Passenger', 'Knee Airbag-Passenger', 'LATERAL CRASH PROTECTION-Passenger', 'Side Head Airbag-Passenger', 'Side Chest Airbag-Passenger', 'Side Pelvis Airbag-Passenger', 'Centre Airbag-Passenger', 'CHILD PROTECTION-Passenger', 'Isofix/i-Size-Passen

In [25]:
df = pd.DataFrame(DATA) # Convert DATA to dataFrame
df.columns = df.iloc[0] # set row 0 as a columns name
df.drop(0, inplace= True) # drop row 0
df.to_csv("Data/CarsRatings.csv") # export the dataset as csv file
df

,Rate,Adult Occupant,Child Occupant,Vulnerable Road Users,Safety Assist,Tested Model,Body Type,Year Of Publication,Kerb Weight,VIN From Which Rating Applies,...,Isofix/i-Size-Rear,Integrated Child Seat-Rear,Airbag cut-off switch-Rear,Seatbelt Reminder-Rear,Active Bonnet,AEB Vulnerable Road Users,AEB Pedestrian - Reverse,AEB Car-to-Car,Speed Assistance,Lane Assist System
1,5,84%,89%,81%,81%,"Jeep Grand Cherokee 2.0 PHEV 'Overland', LHD",- 5 door SUV,2022,2532kg,- all Grand Cherokees,...,Fitted_Standard,Not_Available,Not_Applicable,Fitted_Standard,Not_Available,Fitted_Standard,Not_Available,Fitted_Standard,Fitted_Standard,Fitted_Standard
2,5,87%,87%,68%,88%,"CHERY OMODA5 1.6T, LHD",- SUV,2022,1423kg,- all OMODA5,...,Fitted_Standard,Not_Available,Not_Applicable,Fitted_Standard,Not_Available,Fitted_Standard,Fitted_Standard,Fitted_Standard,Fitted_Standard,Fitted_Standard
3,4,75%,84%,70%,69%,"Ford Puma 1.0 MHEV, 'Titanium', LHD",- 5 door SUV,2022,1334kg,- all Ford Pumas,...,Fitted_Standard,Not_Available,Not_Applicable,Fitted_Standard,Not_Available,Fitted_Standard,Not_Available,Fitted_Standard,Fitted_Standard,Fitted_Standard
4,5,84%,90%,74%,84%,VW Amarok 2.0/Ford Ranger 3.0,- Pick-Up,2022,2434kg,- all double cabin Rangers,...,Fitted_Standard,Not_Available,Not_Applicable,Fitted_Standard,Not_Available,Fitted_Standard,Fitted_Standard,Fitted_Standard,Fitted_Standard,Fitted_Standard
5,5,84%,89%,71%,85%,Discovery Sport,- 5 door SUV,2022,2123kg,- SALCP2FX2PH325732,...,Fitted_Standard,Not_Available,Not_Applicable,Fitted_Standard,Fitted_Standard,Fitted_Standard,Not_Available,Fitted_Standard,Fitted_Standard,Fitted_Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,5,93%,84%,nan,60%,"SEAT Ateca 1.6 diesel, LHD",- 5 door SUV,2016,1300kg,- all Atecas of the specification tested,...,Fitted_Standard,Not_Available,Not_Applicable,Fitted_Standard,Not_Available,nan,nan,nan,Fitted_Standard,Not_Fitted
308,5,96%,84%,nan,68%,"VW Tiguan 2.0 TDI 110 kW, LHD",5 door SUV,2016,1531kg,- all Tiguans of the specification tested,...,Fitted_Standard,Not_Available,Not_Applicable,Fitted_Standard,Fitted_Standard,nan,nan,nan,Not_Fitted,Fitted_Standard
309,3,80%,73%,nan,25%,Suzuki Baleno,5 door hatchback,2016,890kg,- all Balenos,...,Fitted_Standard,Not_Available,Not_Applicable,Fitted_Standard,Not_Available,nan,nan,nan,Not_Fitted,Not_Available
310,4,85%,73%,nan,43%,Suzuki Baleno,- 5 door hatchback,2016,890kg,- all Balenos,...,Fitted_Standard,Not_Available,Not_Applicable,Fitted_Standard,Not_Available,nan,nan,nan,Not_Fitted,Not_Available
